In [1]:
'''
    WARNING CONTROL to display or ignore all warnings
'''
import warnings; warnings.simplefilter('default')     #switch betweeb 'default' and 'ignore'

''' Set debug flag to view extended error messages; else set it to False to turn off debugging mode '''
debug = True

# Load Data

## Establish DB connection

In [2]:
import findspark
findspark.init('/opt/spark/spark-3.2.1-bin-hadoop3.2')
from pyspark.sql import SparkSession

# the Spark session should be instantiated as follows
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.jars", "/opt/spark/spark-3.2.1-bin-hadoop3.2/jars/postgresql-42.3.5.jar") \
    .getOrCreate()
#    .config("spark.jars", "postgresql-42.2.10-1.jar") \
print("Connection complete! ready to load data.")

22/06/05 09:23:31 WARN Utils: Your hostname, AMDFX8520 resolves to a loopback address: 127.0.1.1; using 192.168.1.51 instead (on interface enp2s0)
22/06/05 09:23:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/06/05 09:23:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Connection complete!


## Load and View Schema

In [3]:
print("Wait a moment, retrieving data ...")
load_sdf = spark.read.format("jdbc"). \
options(
    url='jdbc:postgresql://10.11.34.33:5432/Datascience', # jdbc:postgresql://<host>:<port>/<database>
    dbtable='_issuefix_bkdata.customerbookings',
    user='postgres',
    password='postgres',
    driver='org.postgresql.Driver').\
load()
load_sdf.printSchema()

print("Loading complete!")

Wait a moment, retrieving data ...
root
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)
 |-- customer_country: string (nullable = true)
 |-- hotel_name: string (nullable = true)
 |-- hotel_city: string (nullable = true)
 |-- hotel_state: string (nullable = true)
 |-- hotel_country: string (nullable = true)
 |-- checkin_date: date (nullable = true)
 |-- checkout_date: date (nullable = true)
 |-- num_of_nights: integer (nullable = true)
 |-- booked_date: date (nullable = true)
 |-- num_of_rooms: integer (nullable = true)
 |-- daily_rate: double (nullable = true)
 |-- booked_currency: string (nullable = true)

Loading complete!


In [4]:
load_sdf.show(n=2, vertical=True)

-RECORD 0--------------------------------
 customer_city    | Palma                
 customer_state   |                      
 customer_country | Spain                
 hotel_name       | Best Western Plus... 
 hotel_city       | Bryce Canyon         
 hotel_state      | Utah                 
 hotel_country    | USA                  
 checkin_date     | 2021-10-07           
 checkout_date    | 2021-10-09           
 num_of_nights    | 2                    
 booked_date      | 2021-09-11           
 num_of_rooms     | 1                    
 daily_rate       | 338.32               
 booked_currency  | USD                  
-RECORD 1--------------------------------
 customer_city    | Aventura             
 customer_state   |                      
 customer_country | USA                  
 hotel_name       | Hilton Fort Laude... 
 hotel_city       | Fort Lauderdale      
 hotel_state      | Florida              
 hotel_country    | USA                  
 checkin_date     | 2021-05-28    

### Convert to Pandas Dataframe

In [6]:
import pandas as pd

print("Wait a moment while convert loaded data to a dataframe ...")
to_load_pdf = load_sdf.toPandas()
print(to_load_pdf.info())
print(to_load_pdf.head(2))

AttributeError: module 'numpy.random' has no attribute 'BitGenerator'

### Analyse Stats

In [97]:
import sys
sys.path.insert(1, '../wrangler')
import pandas as pd
import stats as ds
import json

#import importlib

if debug:
    import importlib
    ds = importlib.reload(ds)

data_name = "traveller"
cls_stats = ds.DataStatistics(name=data_name)
print(dir(cls_stats))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_dist_types', '_distributions', '_dup_method', '_l_dup_methods', 'count_duplicates', 'count_nulls', 'fit_distributions', 'get_central_tendency', 'get_data_types', 'name', 'test_uniformity']


In [45]:
%%capture
import pandas as pd

stat_summary_df = pd.DataFrame([])
stat_summary_df = cls_stats.fit_distributions(to_load_pdf,["popular"])

In [47]:
print(stat_summary_df)

         distr      score  LLE            loc          scale  \
0   genextreme   0.127618  NaN       1.617076       0.907597   
1        expon   0.129034  NaN            1.0       1.838296   
2       pareto   0.151473  NaN      -0.304145       1.304145   
3        gamma   0.162114  NaN            1.0       4.019531   
4         beta   0.199609  NaN            1.0     176.565283   
5     dweibull   0.243212  NaN       2.527018        1.58273   
6            t   0.264159  NaN       2.404165        1.33507   
7         norm   0.294027  NaN       2.838296       2.092641   
8     loggamma   0.297136  NaN    -551.251433      77.332164   
9      lognorm   0.524361  NaN            1.0       0.000046   
10     uniform   0.542994  NaN            1.0           29.0   
0        expon  13.461062  NaN            1.0       0.030854   
1     dweibull  14.121208  NaN            1.0       0.164265   
2         norm  14.702736  NaN       1.030854       0.221947   
3     loggamma  17.032895  NaN     -76.9

## Drop columns

In [ ]:
to_load_pdf = to_load_pdf.drop('customer_state',axis=1)

## Set data types

In [14]:
print("Wait a moment while we polish the dataframe ...")
#to_load_pdf = pd.DataFrame(list(zip(*_grp_book_collect))).T
#to_load_pdf = to_load_pdf.rename(columns={0 : 'Date', 1 : 'Nights', 2 : 'Rooms', 3 : 'AgentCost'})
to_load_pdf['customer_city'] = to_load_pdf['customer_city'].astype('string')
to_load_pdf['customer_country'] = to_load_pdf['customer_country'].astype('string')
to_load_pdf['hotel_name'] = to_load_pdf['hotel_name'].astype('string')
to_load_pdf['hotel_city'] = to_load_pdf['hotel_city'].astype('string')
to_load_pdf['hotel_state'] = to_load_pdf['hotel_state'].astype('string')
to_load_pdf['booked_date'] = to_load_pdf['booked_date'].astype('datetime64[ns]')
to_load_pdf['checkin_date'] = to_load_pdf['checkin_date'].astype('datetime64[ns]')
to_load_pdf['checkout_date'] = to_load_pdf['checkout_date'].astype('datetime64[ns]')
to_load_pdf['num_of_nights'] = to_load_pdf['num_of_nights'].astype('int64')
to_load_pdf['num_of_rooms'] = to_load_pdf['num_of_rooms'].astype('int64')
to_load_pdf['daily_rate'] = to_load_pdf['daily_rate'].astype('float64')
to_load_pdf['booked_value'] = round(to_load_pdf['num_of_nights']* \
                                to_load_pdf['num_of_rooms']* \
                                to_load_pdf['daily_rate'].astype('float64'),2)
to_load_pdf['booked_currency'] = to_load_pdf['booked_currency'].astype('string')
to_load_pdf.info()


Wait a moment while we polish the dataframe ...
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 259418 entries, 0 to 259417
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   customer_city     259418 non-null  string        
 1   customer_country  259418 non-null  string        
 2   hotel_name        259418 non-null  string        
 3   hotel_city        259418 non-null  string        
 4   hotel_state       259418 non-null  string        
 5   hotel_country     259418 non-null  object        
 6   checkin_date      259418 non-null  datetime64[ns]
 7   checkout_date     259418 non-null  datetime64[ns]
 8   num_of_nights     259418 non-null  int64         
 9   booked_date       259418 non-null  datetime64[ns]
 10  num_of_rooms      259418 non-null  int64         
 11  daily_rate        259418 non-null  float64       
 12  booked_currency   259418 non-null  string        
 13  booked_valu

## Group the data

In [49]:
import numpy as np

grp_by_dt = to_load_pdf.groupby(['checkin_date',
                                 'customer_city',
                                 'customer_country',
                                 'hotel_city',
                                 'hotel_country']). \
                agg({'booked_value': ['sum', 'mean', 'min', 'max', 'std']})
grp_by_dt = grp_by_dt.round(2)
grp_by_dt.columns = ['value_mean', 'value_min', 'value_max', 'value_stdv']
grp_by_dt['value_stdv'] = grp_by_dt['value_stdv'].fillna(0)
grp_by_dt['customer_city_lat'] = np.nan
grp_by_dt['customer_city_lon'] = np.nan
grp_by_dt['hotel_city_lat'] = np.nan
grp_by_dt['hotel_city_lon'] = np.nan
grp_by_dt = grp_by_dt.reset_index()
print(grp_by_dt)

       checkin_date customer_city        hotel_city  value_mean  value_min  \
0        2021-01-01        Dorset         Las Vegas      733.42     620.84   
1        2021-01-01        Dorset          Portland      436.16     436.16   
2        2021-01-01     Grapevine            Austin      154.98     154.98   
3        2021-01-01     Grapevine      Coral Gables      148.37     148.37   
4        2021-01-01     Grapevine         Las Vegas      341.96     240.74   
...             ...           ...               ...         ...        ...   
142119   2023-10-08      Aventura  Lake Buena Vista    12346.20   12346.20   
142120   2023-10-19        Dublin           Orlando    40431.16   40431.16   
142121   2023-10-26   West Sussex           Orlando     8888.64    8888.64   
142122   2023-11-03         Essex           Orlando     1229.52    1229.52   
142123   2023-11-06        Dublin  Lake Buena Vista    31658.00   31658.00   

        value_max  value_stdv  customer_city_lat  customer_city

## Add lat/lon to cities

In [40]:
''' Import the required library '''
from geopy.geocoders import Nominatim

customer_city = grp_by_dt["customer_city"].unique()
hotel_city = grp_by_dt["hotel_city"].unique()
_l_cities = list(set().union(customer_city, hotel_city))

''' Initialize Nominatim API '''
geolocator = Nominatim(user_agent="Traveller")
''' loop through list to allocate lat/lon pairs '''
_l_city_loc = []
_l_city_err = []
for city in _l_cities:
    try:
#        print(city)
        location = geolocator.geocode(city)
        if location != None:
            _l_city_loc.append([city,location.latitude,location.longitude])
        else:
            _l_city_err.append(city)
            raise ValueError("Error: geocode failed on input %s with message" % city)
    except ValueError as error_message:
#        print("Error: geocode failed on input %s with message %s"%(city, error_message))
        print(error_message)
        pass
#print("The latitude of the location is: ", location.latitude)
#print("The longitude of the location is: ", location.longitude)


/usr/lib/python3.8/socket.py:740: ResourceWarning: unclosed <socket.socket fd=63, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('192.168.235.83', 42154), raddr=('140.211.167.100', 443)>
  self._sock = None


In [44]:
import csv

fields = ['City', 'Latitude', 'Longitude'] 
with open('../data/city_locations.csv', 'w') as f:
      
    # using csv.writer method from CSV package
    write = csv.writer(f)
    write.writerow(fields)
    write.writerows(_l_city_loc)

In [77]:
city_loc_df = pd.read_csv("../data/city_locations.csv")
for rowIdx in city_loc_df.index:
    ''' match and assign lat/lon for customer cities '''
    mask = grp_by_dt['customer_city'] == city_loc_df["City"][rowIdx]
    grp_by_dt['customer_city_lat'][mask] = city_loc_df["Latitude"][rowIdx]
    grp_by_dt['customer_city_lon'][mask] = city_loc_df["Longitude"][rowIdx]
    ''' match and assign lat/lon for hotel cities '''
    mask = grp_by_dt['hotel_city'] == city_loc_df["City"][rowIdx]
    grp_by_dt['hotel_city_lat'][mask] = city_loc_df["Latitude"][rowIdx]
    grp_by_dt['hotel_city_lon'][mask] = city_loc_df["Longitude"][rowIdx]

print("Attempt to assign lat/lon to %d" % grp_by_dt.shape[0])

''' drop all cells without lat/lon pairs; i.e; NaN '''
mask = (grp_by_dt['customer_city_lat'].notna() | grp_by_dt['customer_city_lat'])
grp_by_dt = grp_by_dt[mask]
mask = (grp_by_dt['hotel_city_lat'].notna() | grp_by_dt['hotel_city_lon'])
grp_by_dt = grp_by_dt[mask]

print("Successfully assigned lat/lon to %d" % grp_by_dt.shape[0])

''' write to a csv '''
grp_by_dt.to_csv('../data/travel_from_to.csv',sep=',')

Attempt to assign lat/lon to 142122
Successfully assigned lat/lon to 142122


## Where are they from where are they going

In [ ]:
grp_by_dt.read_csv("../data/travel_from_to.csv")

In [84]:
import networkx as nx

g_travel = nx.Graph(name='Travel_from_to_destination') # Simple Digraph
for rowIdx in grp_by_dt.index:
    ''' customer nodes '''
#    g_travel.add_node(grp_by_dt.loc[rowIdx,'customer_city'],
    g_travel.add_node('CC'+str(rowIdx),
                      pos=(grp_by_dt.loc[rowIdx,'customer_city_lat'],
                           grp_by_dt.loc[rowIdx,'customer_city_lon']),
                      city=grp_by_dt.loc[rowIdx,'customer_city'])
    ''' hotel nodes '''
#    g_travel.add_node(grp_by_dt.loc[rowIdx,'hotel_city'],
    g_travel.add_node('HC'+str(rowIdx),
                      pos=(grp_by_dt.loc[rowIdx,'hotel_city_lat'],
                           grp_by_dt.loc[rowIdx,'hotel_city_lon']),
#                      checkin_date=grp_by_dt.loc[rowIdx,'checkin_date'],
                      city=grp_by_dt.loc[rowIdx,'hotel_city'])
    g_travel.add_edge('CC'+str(rowIdx),
                      'HC'+str(rowIdx),
                      checkin_date=grp_by_dt.loc[rowIdx,'checkin_date'],
                      value_min=grp_by_dt.loc[rowIdx,'value_min'],
                      value_max=grp_by_dt.loc[rowIdx,'value_max'],
                      value_stdv=grp_by_dt.loc[rowIdx,'value_stdv'],
                      value_mean=grp_by_dt.loc[rowIdx,'value_mean'],
                     )

In [101]:
import matplotlib.pyplot as plt
#from mpl_toolkits.basemap import Basemap as Basemap
import sys
sys.path.insert(1, '../')
from basemap.packages import basemap as Basemap
%matplotlib inline

plt.figure(figsize = (20,20))
#nx.draw_networkx(g_travel)
node_pos=nx.get_node_attributes(g_travel,'pos')
nx.draw_networkx(g_travel, node_pos,node_size=900, node_color="y", node_shape='o')
#plt.savefig("./images/map_0.png", format = "png", dpi = 300)
plt.show()

KeyboardInterrupt: 

# Redundant (deprecate)

In [4]:
customSchema = "_issuefix_bkdata"
print("Wait a moment, retrieving data ...")
booking_df = spark.read.format("jdbc") \
    .option("url", "jdbc:postgresql://10.11.34.33:5432/Datascience") \
    .option("dbtable", "_issuefix_bkdata.reservations") \
    .option("user", "postgres") \
    .option("password", "postgres") \
    .option("driver", "org.postgresql.Driver") \
    .load()
print("Loaded schema")
booking_df.printSchema()
print("Loading complete!")

Wait a moment, retrieving data ...
Loaded schema
root
 |-- vendor_uuid: string (nullable = true)
 |-- agent_name: string (nullable = true)
 |-- hotel_uuid: string (nullable = true)
 |-- hotel_name: string (nullable = true)
 |-- reserved_status: string (nullable = true)
 |-- reserved_datetime: string (nullable = true)
 |-- booking_for_datetime: date (nullable = true)
 |-- reserved_for_datetime: date (nullable = true)
 |-- check_in_date: date (nullable = true)
 |-- check_out_date: date (nullable = true)
 |-- number_of_nights: integer (nullable = true)
 |-- number_of_rooms: integer (nullable = true)
 |-- booking_type: string (nullable = true)
 |-- agent_booking_currency: string (nullable = true)
 |-- agent_daily_rate: double (nullable = true)
 |-- agent_total_cost: double (nullable = true)
 |-- hotel_currency: string (nullable = true)
 |-- hotel_daily_rate: float (nullable = true)
 |-- group_booking_type: string (nullable = true)

Loading complete!
